In [29]:
import pandas as pd
from tensorflow.keras.models import load_model
from helper.metric import f1_m, recall_m, precision_m
from skimage import io
from helper.face_detection import detect_faces
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img
from PIL import Image
from tqdm import tqdm

tqdm.pandas()

In [30]:
submission = pd.read_csv("../data/submission.csv")
submission

,id
0,005093b2-8c4b-4ed7-91c3-f5f4d50f8d27
1,0052554e-069e-4c43-beb0-0885e8f7684e
2,0092b954-1143-4a95-a17b-1edfa6af3b01
3,009fc28b-fe9b-441d-b8a2-ea8b7ae6ca16
4,00d0e306-06fe-45d8-ae6c-6f83ab8f7810
...,...
985,feb9e70f-4182-4500-866d-9b95657e727c
986,fed67ed2-620a-4ebf-b61e-16ef3d9b93f7
987,ff14c77e-c3c2-46db-9341-30de4130ef8a
988,ff662f3f-8c76-4d27-a73e-c43c4353f798


In [31]:
model = load_model("../model/sequence/gender_classification_model.h5",
                   custom_objects={'f1_m': f1_m, 'recall_m': recall_m, 'precision_m': precision_m})

In [32]:
def predict_gender(image_name):
    try:
        img_path = '../data/Testing/{}.jpg'
        image = io.imread(img_path.format(image_name))
        detected_faces = detect_faces(image)
        face_target = Image.fromarray(image).crop(detected_faces[0])
        h, w = face_target.size
        img = img_to_array(face_target)
        img = img / 255
        img = img.reshape(1, h, w, 3)
        prob = model.predict(img)[0][0]
        if prob <= 0.5:
            return int(0)
        else:
            return int(1)
    except IndexError:
       return int(1)
       pass


In [33]:
submission.iloc[0]["id"]

'005093b2-8c4b-4ed7-91c3-f5f4d50f8d27'

In [34]:
predict_gender(submission.iloc[0]["id"])

2021-10-27 05:59:53.328362: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


0

In [35]:
submission["jenis kelamin"] = submission["id"].progress_apply(predict_gender)
submission

100%|██████████| 990/990 [15:18<00:00,  1.08it/s]


,id,jenis kelamin
0,005093b2-8c4b-4ed7-91c3-f5f4d50f8d27,0
1,0052554e-069e-4c43-beb0-0885e8f7684e,0
2,0092b954-1143-4a95-a17b-1edfa6af3b01,0
3,009fc28b-fe9b-441d-b8a2-ea8b7ae6ca16,0
4,00d0e306-06fe-45d8-ae6c-6f83ab8f7810,1
...,...,...
985,feb9e70f-4182-4500-866d-9b95657e727c,0
986,fed67ed2-620a-4ebf-b61e-16ef3d9b93f7,0
987,ff14c77e-c3c2-46db-9341-30de4130ef8a,1
988,ff662f3f-8c76-4d27-a73e-c43c4353f798,1


In [36]:
submission.isnull().sum()

id               0
jenis kelamin    0
dtype: int64

In [38]:
submission.to_csv("../data/result/submission.csv", index=False)